<div style="text-align: center;">
    <h1><b>SCHICHTPLAN V. 2.0</b></h1>
    <p>Francisco Javier Cabello Rueda | Rewe Digital | 20/03/2025</p>
    <img src="./img/rewe.png" alt="Texto alternativo" width="400">
</div>


<div style="text-align: center;">
    <h2><b>TABLE OF CONTENTS</b></h2>
</div>

1. Project Structure <br>
2. Server Preparation <br>
3. Database Backup <br>
4. Employees Actions <br>
5. Shifts Actions <br>
6. Languages <br>

<div>
    <h2 style="text-align: center;"><b>1º PROJECT STRUCTURE</b></h2>
    <ul>
        <li>As a purpose of the project we want to develop an application to manage the shift plan. For this we will create a home page where we can choose in which database we want to act and which action we want to perform, among them are:</li>
        <br>
        <li>Create Employee
    - Delete Employee
- Create Shift
- Delete Shift
- Backup of the database</li>
    </ul>
</div>
<div style="text-align: center;">
    <img src="./img/php.jpg" alt="Texto alternativo" width="400">
</div>

<div>
    <h2 style="text-align: center;"><b>2º SERVER PREPARATION</b></h2>
    <ul>
        <li>
            First, we need to access the Linux server where we will host the Schcichtplan, for this we will access the Rewe PAM and log in (
            <a href="https://pam.rewe.local">https://pam.rewe.local</a>).
        </li>
        <br>
        <li>
            Once you have logged in, click on connect via SSH and enter the address of the server.
        </li>
        <br>
        <div style="text-align: center;">
            <img src="./img/pam.png" alt="Texto alternativo" width="300">
        </div>
        <br>
        <li>
            When we have entered the server we will proceed to its configuration, first we will do a <code>sudo dnf upgrade</code> to keep the server up to date and we will install php by <code>sudo dnf install php</code>.
        </li>
        <br>
        <li>
            Now we will proceed to the installation of <code>mod_ssl</code> which will provide us with support for SSL connections, for this we will run a <code>sudo dnf list mod_ssl</code> and <code>sudo dnf install mod_ssl</code>.
        </li>
        <br>
        <li>
            Then we will create a key with OpenSSL that will be used to create a certificate, for the creation of the key we will use the instruction <code>openssl req -new -nodes -sha256 -out leitstand-test.rewe.local.csr -config SAN_CERT.conf -keyout leitstand-test.rewe.local.key</code> that then we will copy and paste in this web to create the certificate (ticket system of rewe).
        </li>
        <br>
        <li>
            To continue we will configure the <code>SSL.conf</code> file to create a virtualhost on port 443, to do this we will go to the directory <code>/etc/httpd/conf.d</code> and in this we will do a <code>sudo nano ssl.conf</code>. Once inside we will look for the directives <code>SSLCertificateFile</code> and <code>SSLCertificateKeyFile</code> and we will put the path in which they are hosted, this is usually <code>/etc/ssl</code>. Then we will configure <code>httpd.conf</code> to create a virtualhost on port 80. The crt-file will be empty, so we will go to the Ticket and download the crt-file, open it in Notepad, copy the link and paste it into the crt-file on our server. To format it we will launch the commands <code>sed -i -e ‘s/.\{64}/&&n/g’</code> and <code>sed -i ‘/^$/d’</code>. Once configured, save the changes and perform a <code>sudo systemctl restart httpd</code> to update the changes. With this we will have the SSL configuration ready.
        </li>
    </ul>
</div>

<h3 style="text-align: center;"><b>ssl.conf</b></h3>

In [ ]:
#
# /etc/httpd/conf.d/ssl.conf
#

Listen 443 https

SSLPassPhraseDialog exec:/usr/libexec/httpd-ssl-pass-dialog


SSLSessionCache         shmcb:/run/httpd/sslcache(512000)
SSLSessionCacheTimeout  300

SSLCryptoDevice builtin

<VirtualHost *:443>

DocumentRoot "/var/www/html"
ServerName leitstand-test.rewe.local:443
ErrorLog logs/ssl_error_log
TransferLog logs/ssl_access_log
LogLevel warn

SSLEngine on

SSLHonorCipherOrder on

SSLCipherSuite PROFILE=SYSTEM
SSLProxyCipherSuite PROFILE=SYSTEM


SSLCertificateFile /etc/httpd/cert/leitstand-test.rewe.local.crt

SSLCertificateKeyFile /etc/httpd/cert/leitstand-test.rewe.local.key

SSLCertificateChainFile /etc/httpd/cert/deviceca05.crt

<FilesMatch "\.(cgi|shtml|phtml|php)$">
    SSLOptions +StdEnvVars
</FilesMatch>
<Directory "/var/www/cgi-bin">
    SSLOptions +StdEnvVars
</Directory>

BrowserMatch "MSIE [2-5]" \
         nokeepalive ssl-unclean-shutdown \
         downgrade-1.0 force-response-1.0

CustomLog logs/ssl_request_log \
          "%t %h %{SSL_PROTOCOL}x %{SSL_CIPHER}x \"%r\" %b"

</VirtualHost>



<h3 style="text-align: center;"><b>httpd.conf</b></h3>

In [ ]:
#
# /etc/httpd/conf/httpd.conf
#

ServerRoot "/etc/httpd"

Listen 80

<VirtualHost *:80>
    ServerName leitstand-test.rewe.local
    Redirect permanent / https://leitstand-test.rewe.local/
</VirtualHost>

Include conf.modules.d/*.conf

User apache
Group apache

ServerAdmin root@localhost

<Directory />
    AllowOverride none
    Require all denied
</Directory>

DocumentRoot "/var/www/html"

<Directory "/var/www">
    AllowOverride None
    Require all granted
</Directory>

<Directory "/var/www/html">
    Options Indexes FollowSymLinks
    AllowOverride None
    Require all granted
</Directory>

<IfModule dir_module>
    DirectoryIndex index.html
</IfModule>

<Files ".ht*">
    Require all denied
</Files>


ErrorLog "logs/error_log"
LogLevel warn

<IfModule log_config_module>
    LogFormat "%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"" combined
    LogFormat "%h %l %u %t \"%r\" %>s %b" common
    <IfModule logio_module>
      LogFormat "%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\" %I %O" combinedio
    </IfModule>
    CustomLog "logs/access_log" combined
</IfModule>

<IfModule alias_module>
    ScriptAlias /cgi-bin/ "/var/www/cgi-bin/"
</IfModule>

<Directory "/var/www/cgi-bin">
    AllowOverride None
    Options None
    Require all granted
</Directory>

<IfModule mime_module>
    TypesConfig /etc/mime.types
    AddType application/x-compress .Z
    AddType application/x-gzip .gz .tgz
    AddType text/html .shtml
    AddOutputFilter INCLUDES .shtml
</IfModule>

AddDefaultCharset UTF-8

<IfModule mime_magic_module>
    MIMEMagicFile conf/magic
</IfModule>

EnableSendfile on

IncludeOptional conf.d/*.conf






<div>
    <ul>
        <li>
            in case the ssl does not work, check that all relevant files have <code>755 permissions</code> or if the <code>certificate</code> is well created, it is possible that the creation instruction is wrong.
        </li><br>
        <li>
            To finish with the preparation of the server we will create an index.php where we can select what action we want to perform and on which server we want to perform it and a file that manages the logic of selection of the server.
        </li>
    </ul>
    <h3 style="text-align: center;"><b>Index.php</b></h3>
</div>


In [ ]:
<!-- Designed by Francisco Javier Cabello Rueda on 20/05/2025  -->

<!DOCTYPE html>
<html lang="en">
	<head>
		<meta http-equiv="cache-control" content="no-cache, no-store, must-revalidate">
		<meta http-equiv="pragma" content="no-cache"><meta http-equiv="expires" content="0">
		<meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0">
		<link rel="icon" type="image/png" sizes="32x32" href="assets/favicon/rewe.png">
		<title>Schichtplan | Home</title>
		<link rel="stylesheet" href="assets/css/_header.css">
		<link rel="stylesheet" href="assets/css/_footer.css">
		<link rel="stylesheet" href="assets/css/_font.css">
		<link rel="stylesheet" href="assets/css/_content.css">
	</head>
	<body>
		<?php
			// Start a new session or resume the existing session
			session_start();
			// Check if the request method is POST, which indicates form submission
			if ($_SERVER['REQUEST_METHOD'] === 'POST') {
				// Check if the 'dbEnvironment' key is present in POST data
				if (isset($_POST['dbEnvironment'])) {
					// Set the 'dbEnvironment' session variable to the value submitted in the form
					$_SESSION['dbEnvironment'] = $_POST['dbEnvironment'];
				}
			}
			// If no 'dbEnvironment' is set in the session, initialize it with a default value 'test'
			if (!isset($_SESSION['dbEnvironment'])) {
				$_SESSION['dbEnvironment'] = 'test';
			}
		?>
		<?php
			echo file_get_contents("header.html");
    	?>
		<div class="content">
			<h2>Home</h2>
			<div class="container drop-shadow-m">
				<div class="line"></div>
				<p class="label">Welcome to the Shift Plan!</p>
				<p class="label">Please select the database environment where you want to work:</p>
				<form method="post">
					<label><input type="radio" name="dbEnvironment" value="test" <?php echo $_SESSION['dbEnvironment'] === 'test' ? 'checked' : ''; ?>> Test </label>
					<label><input type="radio" name="dbEnvironment" value="dev" <?php echo $_SESSION['dbEnvironment'] === 'dev' ? 'checked' : ''; ?>> Dev </label>
					<label><input type="radio" name="dbEnvironment" value="prod" <?php echo $_SESSION['dbEnvironment'] === 'prod' ? 'checked' : ''; ?>> Prod </label>
					<button type="submit" style="margin-left: 10px">Save Environment</button>
				</form>
				<p>Please select one of the options below:</p>
				<div class="section">
					<button onclick="window.location.href='create-employee.php'">Create employee</button><br>
					<button onclick="window.location.href='remove-employee.php'">Remove employee</button><br>
					<button onclick="window.location.href='create-shift.php'">Create shift</button><br>
					<button onclick="window.location.href='remove-shift.php'">Remove shift</button><br>
					<button onclick="window.location.href='backup-database.php'">Backup database</button><br><br>
					<button onclick="window.location.href='health-database.php'">Health database</button><br><br>
				</div>
				<p class="label">v. 2.0</p>
			</div>
		</div>
		<?php
    		echo file_get_contents("footer.html");
    	?>
	</body>
</html>

 <h3 style="text-align: center;"><b>dbconfig.php</b></h3>

In [ ]:
<!-- Designed by Francisco Javier Cabello Rueda on 20/05/2025  -->

<?php
// Start a new session or resume the existing session
session_start();

// Determine the database environment from the session
// Defaulting to 'test' if no environment has been set
$environment = $_SESSION['dbEnvironment'] ?? 'test';

// Define connection details based on the selected environment
$dbDetails = [
    'test' => [
        'host' => 'hcvm101953.linux.risnet.de',
        'port' => '5432',
        'user' => 'dbpo_sbuchuser',
        'password' => 'TST!BbLEN7VW744gBoRMxZsHEa4nNd5uo2AweKN',
        'dbname' => 'sbuch_tst',
    ],
    'dev' => [
        'host' => 'hcvm102705.linux.risnet.de',
        'port' => '5432',
        'user' => 'dbpo_sbuchuser',
        'password' => 'TST!BbLEN7VW744gBoRMxZsHEa4nNd5uo2AweKN',
        'dbname' => 'sbuch_dev',
    ],
    'prod' => [
        'host' => 'vvm10693.de.rewe.local',
        'port' => '5432',
        'user' => 'SBuchuser_dbo',
        'password' => 'VhhDETgW9gXU',
        'dbname' => 'SbuchP',
    ],
];

// Set up the connection details for the current environment
$host = $dbDetails[$environment]['host'];
$port = $dbDetails[$environment]['port'];
$user = $dbDetails[$environment]['user'];
$password = $dbDetails[$environment]['password'];
$dbname = $dbDetails[$environment]['dbname'];

// Create a connection string using the environment-specific details
$conn_string = "host=$host port=$port dbname=$dbname user=$user password=$password";

// Attempt to establish a connection to the PostgreSQL database
$conn = pg_connect($conn_string);

// Check if the connection was successful; if not, terminate the script with an error message
if (!$conn) {
    die("Connection failed: " . pg_last_error());
}

// Set client encoding to UTF-8 for the current connection
pg_set_client_encoding($conn, "UTF8");

// Setup the schema search path
pg_query($conn, "SET search_path TO sbuchp");
?>

<div>
    <h2 style="text-align: center;"><b>3º DATABASE BACKUP</b></h2>
    <ul>
        <li>
            Now that we have finished the configuration of our server we are going to proceed to make a php code that allows us to backup a database and migrate that information to another database.
        </li>
        <br>
        <li>
            Before we start we have to verify that we can establish connection with the databases. First we will make sure that we can make backups from pgadmin, for this we will connect the original database and the database where we want to make the copy, we will right click on the original and we will give backup, creating an sql file, with this we go to the new database and we give restore, if it is done without problem we can start testing with php. 
        </li>
        <br>
        <li>
            In case of some problem it can be due to a problem of versions with pgadmin, to solve it inside pgadmin, we go to File/Preferences/Paths/Binary-Paths and inside here we will put the path of posgresql in its respective version, in my case C:\Program Files\PostgreSQL\15\bin. 
        </li>
        <br>
        <li>
            Then we will create a basic php code that makes connection with the databases and shows if everything goes correctly or there is an error. In case of error there may be a firewall enabled preventing the connection or the SELinux that can be disabled with <code>setenforce 0</code>.
        </li>
        <br>
    </ul>
     <h3 style="text-align: center;"><b>health-database.php</b></h3>
</div>

In [ ]:
<!-- Designed by Francisco Javier Cabello Rueda on 22/05/2025  -->

<!DOCTYPE html>
<html lang="en">
    <head>
        <meta http-equiv="cache-control" content="no-cache, no-store, must-revalidate">
        <meta http-equiv="pragma" content="no-cache">
        <meta http-equiv="expires" content="0">
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <link rel="icon" type="image/png" sizes="32x32" href="assets/favicon/rewe.png">
        <title>Schichtplan | Health-Database</title>
        <link rel="stylesheet" href="assets/css/_header.css">
        <link rel="stylesheet" href="assets/css/_footer.css">
        <link rel="stylesheet" href="assets/css/_font.css">
        <link rel="stylesheet" href="assets/css/_content.css">
    </head>
    <body>
        <?php 
            echo file_get_contents("header.html"); 
        ?>
        <div class="content">
            <h2 class="section-title">Health Database</h2>
            <div class="container drop-shadow-m">
                <div class="line"></div>
                <?php
                    session_start();

                    // Check session state for database environment
                    $environment = $_SESSION['dbEnvironment'] ?? 'test';

                    // Define configurations for `test`, `dev`, and `prod` environments
                    $dbConfigs = [
                        'test' => [
                            'host' => 'hcvm101953.linux.risnet.de',
                            'port' => '5432',
                            'user' => 'dbpo_sbuchuser',
                            'dbname' => 'sbuch_tst',
                            'password' => 'TST!BbLEN7VW744gBoRMxZsHEa4nNd5uo2AweKN',
                        ],
                        'dev' => [
                            'host' => 'hcvm102705.linux.risnet.de',
                            'port' => '5432',
                            'user' => 'dbpo_sbuchuser',
                            'dbname' => 'sbuch_dev',
                            'password' => 'TST!BbLEN7VW744gBoRMxZsHEa4nNd5uo2AweKN',
                        ],
                        'prod' => [
                            'host' => 'vvm10693.de.rewe.local',
                            'port' => '5432',
                            'user' => 'SBuchuser_dbo',
                            'dbname' => 'SbuchP',
                            'password' => 'VhhDETgW9gXU',
                        ],
                    ];

                    // Loop through each environment for connection and display results
                    foreach (['test', 'dev', 'prod'] as $env) {
                        $config = $dbConfigs[$env];

                        // Create connection string for the current environment
                        $connString = "host={$config['host']} port={$config['port']} dbname={$config['dbname']} user={$config['user']} password={$config['password']}";

                        // Attempt to connect to the PostgreSQL database
                        $conn = pg_connect($connString);

                        // Output results
                        echo '<div class="section">';
                        if ($conn) {
                            echo "<div class='status-box alert-success'>Succesfully connected to the <b>{$env}</b> database!</div>";
                            pg_close($conn);
                        } else {
                            echo "<div class='status-box alert-danger'>Error connecting to the <b>{$env}</b> database!</div>";
                            error_log("Error connecting to the {$env} database.");
                        }
                        echo '</div>';
                    }
                ?>
            </div>
        </div>
        <?php 
            echo file_get_contents("footer.html"); 
        ?>
    </body>
</html>

    
<ul>    
    <li>
        Now that we have connection we can create the code to make the backup. First we will create the code that will allow us to choose the source and target database, as well as show us a tracking of the backup process via ajax. Then we will create the code with the logic to perform the backup.
    </li>
    <br>
    <li>
        It is possible that if the database we want to export is too big we can't add it to the new database, to change it we will modify <code>upload_max_filesize</code> and <code>post_max_size</code> to 60M and <code>max_execution_time</code> and <code>max_input_time</code> to 300 in <code>/etc/php.ini</code>, to apply the changes we do a <code>sudo systemctl restart php-fpm</code>. We will also need to add <code>Timeout 300</code> in <code>/etc/httpd/conf/httpd.conf</code>, to apply the changes we launch a <code>sudo systectl restart httpd</code>.
        If everything works correctly we can go to the new database and notice that it is now populated with data.
    </li>
</ul>

<h3 style="text-align: center;"><b>backup-database.php</b></h3>
<ul>    
    <li>
        This code allows users to select source and target databases for a backup. When the "Run Backup" button is clicked, it sends the database selection to a server script via AJAX and displays real-time progress in a container.
    </li>
</ul>

In [ ]:
<!-- Designed by Francisco Javier Cabello Rueda on 22/05/2025  -->
 
<!DOCTYPE html>
<html lang="en">
	<head>
		<meta http-equiv="cache-control" content="no-cache, no-store, must-revalidate">
		<meta http-equiv="pragma" content="no-cache">
		<meta http-equiv="expires" content="0">
		<meta charset="utf-8">
		<meta name="viewport" content="width=device-width, initial-scale=1.0">
		<link rel="icon" type="image/png" sizes="32x32" href="assets/favicon/rewe.png"/>
		<title>Schichtplan | Backup-Database</title>
		<link rel="stylesheet" href="assets/css/_header.css">
		<link rel="stylesheet" href="assets/css/_content.css">
		<link rel="stylesheet" href="assets/css/_footer.css">
		<link rel="stylesheet" href="assets/css/_font.css">
		<style>
			.progress-container {
				border: 1px solid #ccc;
				padding: 10px;
				width: 100%;
				height: 200px;
				overflow-y: scroll;
			}
		</style>
	</head>
	<body>
		<?php
			echo file_get_contents ("header.html");
		?>
		<div class="content">
			<h2>Backup Database</h2>
			<div class="container drop-shadow-m">
				<div class="line"></div>
				<form method="post" id="backupForm">	
					<div class="section">
						<p class="label">Select the source and target databases and execute backup.</p> <br>
					</div>
					<div class="section">
						<span>Source Database: 
							<input type="radio" class="employee-checkbox" name="dbSource" value="prod" required>
							<label for="prod">Prod</label>
							<input type="radio" class="employee-checkbox" name="dbSource" value="test" required>
							<label for="test">Test</label>
						</span>	
					</div>
					<div class="section">
						<span>Target Database: 
							<input type="radio" class="employee-checkbox" name="dbTarget" value="dev" required>
							<label for="dev">Dev</label>
							<input type="radio" class="employee-checkbox" name="dbTarget" value="test" required>
							<label for="test">Test</label>
						</span>	
					</div>	
					<div class="section">		
						<button type="button" class="button-primary" onclick="startBackup()">Run Backup</button>
					</div>
				</form>
				<div class="progress-container" id="progress-container"></div>
			</div>
		</div>
		<?php
			echo file_get_contents ("footer.html");
		?>
		<script>
			// Get the HTML element where progress will be displayed
			const progressContainer = document.getElementById('progress-container');
			// Function to start the backup process
			function startBackup() {
				// Create a FormData object containing the data from the form with id 'backupForm'
				const formData = new FormData(document.getElementById('backupForm'));
				// Create a new XMLHttpRequest object for making HTTP requests
				const xhr = new XMLHttpRequest();
				// Initialize a POST request to the 'backup.php' script, and keep it asynchronous
				xhr.open('POST', 'backup-script.php', true);
				// Define a function to be called whenever the readyState changes
				xhr.onreadystatechange = function () {
					// If the request is in loading state or done state
					if (xhr.readyState === XMLHttpRequest.LOADING || xhr.readyState === XMLHttpRequest.DONE) {
						// Append the response text to the progress container, replacing newline characters with <br/>
						progressContainer.innerHTML += xhr.responseText.replace(/\n/g, '<br/>');

						// Scroll to the bottom of the progress container to show the latest updates
						progressContainer.scrollTop = progressContainer.scrollHeight;
					}
				};
				// Send the form data using the XMLHttpRequest
				xhr.send(formData);
			}
		</script>
	</body>
</html>


<h3 style="text-align: center;"><b>backup-script.php</b></h3>


<ul>    
    <li>
       This PHP script connects to two selected PostgreSQL databases, prepares the target schema, transfers data from source tables, and provides real-time progress updates.
    </li>
</ul>



In [ ]:
<!-- Designed by Francisco Javier Cabello Rueda on 20/05/2025  -->

<?php
// Fetch the selected databases
$dbSource = $_POST['dbSource'] ?? 'test';
$dbTarget = $_POST['dbTarget'] ?? 'test';

// Define connection details based on selections
if ($dbSource === 'test') {
    $dbHostSource = 'hcvm101953.linux.risnet.de';
    $dbPortSource = '5432';
    $dbNameSource = 'sbuch_tst';
    $dbUserSource = 'dbpo_sbuchuser';
    $dbPasswordSource = 'TST!BbLEN7VW744gBoRMxZsHEa4nNd5uo2AweKN';
} elseif ($dbSource === 'prod') {
    $dbHostSource = 'prod_host';
    $dbPortSource = '5432';
    $dbNameSource = 'prod_name';
    $dbUserSource = 'prod_user';
    $dbPasswordSource = 'prod_pass';
}

if ($dbTarget === 'test') {
    $dbHostTarget = 'hcvm101953.linux.risnet.de';
    $dbPortTarget = '5432';
    $dbNameTarget = 'sbuch_tst';
    $dbUserTarget = 'dbpo_sbuchuser';
    $dbPasswordTarget = 'TST!BbLEN7VW744gBoRMxZsHEa4nNd5uo2AweKN';
} elseif ($dbTarget === 'dev') {
    $dbHostTarget = 'hcvm102705.linux.risnet.de';
    $dbPortTarget = '5432';
    $dbNameTarget = 'sbuch_dev';
    $dbUserTarget = 'dbpo_sbuchuser';
    $dbPasswordTarget = 'TST!BbLEN7VW744gBoRMxZsHEa4nNd5uo2AweKN';
}

// Function to send updates to the client
function sendUpdate($message, $color) {
    // Send message with HTML color styling
    echo "<span style='color:$color;'>$message</span><br>";
    echo str_repeat(' ', 4096); // Ensure immediate sending
    flush();
    usleep(500000); // Wait for 0.5 seconds between sends
}

// Set headers for content-type and cache-control
header('Content-Type: text/event-stream');
header('Cache-Control: no-cache');

if(!isset($_POST['dbSource']) || !isset($_POST['dbTarget'])) {
    sendUpdate("Please select both source and target databases.", "#B6424C");
    exit();
}

if($dbSource == $dbTarget) {
    sendUpdate("You cannot export a database by itself.", "#B6424C");
    exit();
}
// Connect to the source database
$connSourceString = "host=$dbHostSource port=$dbPortSource dbname=$dbNameSource user=$dbUserSource password=$dbPasswordSource";
$connSource = pg_connect($connSourceString);

if (!$connSource) {
    sendUpdate("Error connecting to the source database.", "#B6424C");
    error_log("Error connecting to the source database.");
    exit();
}

sendUpdate("Connected to the source database successfully.", "#000000");

// Connect to the target database
$connTargetString = "host=$dbHostTarget port=$dbPortTarget dbname=$dbNameTarget user=$dbUserTarget password=$dbPasswordTarget";
$connTarget = pg_connect($connTargetString);

if (!$connTarget) {
    sendUpdate("Error connecting to the target database.", "#B6424C");
    error_log("Error connecting to the target database.");
    exit();
}

sendUpdate("Connected to the target database successfully.", "#000000");

// Create the schema sbuchp if it does not exist
$createSchemaQuery = "CREATE SCHEMA IF NOT EXISTS sbuchp";
$result = pg_query($connTarget, $createSchemaQuery);
if (!$result) {
    sendUpdate("Error creating schema sbuchp: " . pg_last_error($connTarget), "#B6424C");
    exit();
}

sendUpdate("Schema sbuchp created/existing.", "#006400");

// Set the search path to the schema sbuchp
pg_query($connTarget, "SET search_path TO sbuchp");

// Remove all tables from the sbuchp schema in the target database
$dropTablesQuery = "
    DO $$ DECLARE r RECORD;
    BEGIN
        FOR r IN (SELECT tablename FROM pg_tables WHERE schemaname = 'sbuchp') LOOP
            EXECUTE 'DROP TABLE IF EXISTS sbuchp.' || quote_ident(r.tablename) || ' CASCADE';
        END LOOP;
    END $$;";
$result = pg_query($connTarget, $dropTablesQuery);
if (!$result) {
    sendUpdate("Error dropping tables: " . pg_last_error($connTarget), "#B6424C");
    pg_close($connTarget);
    exit();
}

sendUpdate("Tables dropped from schema sbuchp in the target database.", "#006400");

// Begin a transaction
pg_query($connTarget, "BEGIN");

// Query all tables from the sbuchp schema in the source database
$tablesResult = pg_query($connSource, "SELECT table_name FROM information_schema.tables WHERE table_schema = 'sbuchp'");
if (!$tablesResult) {
    sendUpdate("Error retrieving table names.", "#B6424C");
    pg_close($connSource);
    pg_close($connTarget);
    exit();
}

sendUpdate("Table names retrieved successfully.", "#000000");

// Process each table from the source database
while ($tableRow = pg_fetch_assoc($tablesResult)) {
    $tableName = $tableRow['table_name'];
    
    sendUpdate("Processing table: $tableName.", "#000000");
    
    // Query primary keys for the current table
    $pkQuery = "
        SELECT a.attname AS column_name
        FROM pg_constraint AS c
        JOIN pg_attribute AS a ON a.attnum = ANY(c.conkey) AND a.attrelid = c.conrelid
        WHERE c.conrelid = 'sbuchp.$tableName'::regclass AND c.contype = 'p'";
    $pkResult = pg_query($connSource, $pkQuery);
    if (!$pkResult) {
        sendUpdate("Error retrieving primary keys for table $tableName.", "#B6424C");
        continue;
    }
    
    $primaryKeys = [];
    while ($pkRow = pg_fetch_assoc($pkResult)) {
        $primaryKeys[] = $pkRow['column_name'];
    }
    
    // Query foreign keys for the current table
    $foreignKeysMap = [];
    $fkQuery = "
        SELECT c.conname AS constraint_name, a.attname AS column_name,
               cf.relname AS foreign_table, af.attname AS foreign_column
        FROM pg_constraint AS c
        JOIN pg_attribute AS a ON a.attnum = ANY(c.conkey) AND a.attrelid = c.conrelid
        JOIN pg_class AS cf ON c.confrelid = cf.oid
        JOIN pg_attribute AS af ON af.attnum = ANY(c.confkey) AND af.attrelid = c.confrelid
        WHERE c.conrelid = 'sbuchp.$tableName'::regclass AND c.contype = 'f'";
    $fkResult = pg_query($connSource, $fkQuery);
    if (!$fkResult) {
        sendUpdate("Error retrieving foreign keys for table $tableName.", "#B6424C");
        continue;
    }
    
    while ($fkRow = pg_fetch_assoc($fkResult)) {
        $foreignKeysMap[$fkRow['column_name']][] = "\"{$fkRow['column_name']}\" REFERENCES sbuchp.\"{$fkRow['foreign_table']}\"(\"{$fkRow['foreign_column']}\")";
    }
    
    // Query column details including type and constraints
    $columnsQuery = "
        SELECT column_name, data_type, character_maximum_length, numeric_precision, numeric_scale, is_nullable
        FROM information_schema.columns
        WHERE table_schema = 'sbuchp' AND table_name = '$tableName'";
    $columnsResult = pg_query($connSource, $columnsQuery);
    if (!$columnsResult) {
        sendUpdate("Error retrieving column information for table $tableName.", "#B6424C");
        continue;
    }
    
    $columns = [];
    while ($columnRow = pg_fetch_assoc($columnsResult)) {
        $columnName = $columnRow['column_name'];
        $columnType = strtoupper($columnRow['data_type']);
        $maxLength = $columnRow['character_maximum_length'];
        $numericPrecision = $columnRow['numeric_precision'];
        $numericScale = $columnRow['numeric_scale'];
        $isNullable = $columnRow['is_nullable'] === 'YES' ? '' : ' NOT NULL';
        $isPrimaryKey = in_array($columnName, $primaryKeys) ? ' PRIMARY KEY' : '';

        $typeDetails = $columnType;
        if (strpos($columnType, 'CHAR') !== false && $maxLength) {
            $typeDetails .= "($maxLength)";
        } elseif (strpos($columnType, 'NUMERIC') !== false && $numericPrecision) {
            $typeDetails .= $numericScale ? "($numericPrecision, $numericScale)" : "($numericPrecision)";
        }
        
        $foreignKeyStr = isset($foreignKeysMap[$columnName]) ? implode(' ', $foreignKeysMap[$columnName]) : '';

        $columns[] = "\"$columnName\" $typeDetails$isPrimaryKey$isNullable $foreignKeyStr";
    }
    
    // Create table in target database
    if (!empty($columns)) {
        $createTableQuery = "CREATE TABLE sbuchp.\"$tableName\" (" . implode(', ', $columns) . ")";
        $result = pg_query($connTarget, $createTableQuery);
        if (!$result) {
            sendUpdate("Error creating table sbuchp.$tableName: " . pg_last_error($connTarget), "#B6424C");
            pg_query($connTarget, "ROLLBACK");
            exit();
        }
        
        sendUpdate("Table '$tableName' created in schema sbuchp.", "#006400");
    } else {
        sendUpdate("No columns found in table '$tableName'.", "#B6424C");
        continue;
    }
    
    // Source data from source database to target database
    $query = "SELECT * FROM sbuchp.\"$tableName\"";
    $result = pg_query($connSource, $query);
    if (!$result) {
        sendUpdate("Error Sourceing data from table $tableName.", "#B6424C");
        continue;
    }
    
    while ($row = pg_fetch_assoc($result)) {
        $columnNames = implode(', ', array_keys($row));
        $values = implode(', ', array_map(fn($value) => is_null($value) ? 'NULL' : ("'" . pg_escape_string($connTarget, $value) . "'"), array_values($row)));
        
        $insertQuery = "INSERT INTO sbuchp.\"$tableName\" ($columnNames) VALUES ($values)";
        $insertResult = pg_query($connTarget, $insertQuery);
        if (!$insertResult) {
            sendUpdate("Error inserting into table sbuchp.$tableName: " . pg_last_error($connTarget), "#B6424C");
            exit();
        }
    }
    
    sendUpdate("Data inserted into table '$tableName'.", "#006400");
}

// Commit the transaction
pg_query($connTarget, "COMMIT");

sendUpdate("DATABASE TRANSFER COMPLETED SUCCESSFULLY!!!", "#008000");

// Close database connections
pg_close($connSource);
pg_close($connTarget);

?>

<div>
    <h2 style="text-align: center;"><b>4º EMPLOYEES ACTIONS</b></h2>
    <ul>
        <li>
            Now we will proceed to create the codes that will allow us to create and delete employees from the database.
        </li>
        <br>
    </ul>
    <h3 style="text-align: center;"><b>create-employee.php</b></h3>
    <ul>    
        <li>
        This PHP page allows users to create an employee by entering their first and last name, then inserts this data into a PostgreSQL database with real-time feedback.
        </li>
    </ul>
</div>

In [ ]:
<!-- Designed by Francisco Javier Cabello Rueda on 19/05/2025  -->
 
<?php
  require_once "config/dbconfig.php";
?>

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta http-equiv="cache-control" content="no-cache, no-store, must-revalidate">
    <meta http-equiv="pragma" content="no-cache">
    <meta http-equiv="expires" content="0">
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
	<link rel="icon" type="image/png" sizes="32x32" href="assets/favicon/rewe.png"/>
    <title>Schichtplan | Create-Employee</title>
	<link rel="stylesheet" href="assets/css/_header.css">
	<link rel="stylesheet" href="assets/css/_content.css">
	<link rel="stylesheet" href="assets/css/_footer.css">
	<link rel="stylesheet" href="assets/css/_font.css">
  </head>
  <body>
	<?php
		echo file_get_contents ("header.html");
	?>
	<div class="content">
		<h2>Create Employee</h2>
		<div class="container drop-shadow-m">
			<div class="line"></div>
			<form method="post">			
				<div class="section">
					<p class="label">Enter a first and last name to add an employee.</p> <br>
				</div>	
				<div class="section">
					<input type="text" placeholder="First name" name="firstname" class="textfield" required>
					<input type="text" placeholder="Last name" name="lastname" class="textfield" required>	
				</div>
				<div class="section">
					<input type="submit" name="submit_btn" class="button-primary" value="Create employee">
				</div>
			</form>
			<?php
				// Check if the submit button has been clicked in the form
				if (isset($_POST['submit_btn'])) {
					// Retrieve the 'firstname' and 'lastname' inputs from the POST request
					$firstname = $_POST['firstname'];
					$lastname = $_POST['lastname'];

					// Call the function to create a new employee with the provided data
					createEmployee($firstname, $lastname);
				}

				// Function to insert a new employee record into the database
				function createEmployee($firstname, $lastname) {
					global $conn; // Use the global database connection

					// Find the highest current value of ma_inc and increment it
					$maxQuery = "SELECT MAX(ma_inc) FROM t_mitarbeiter";
					$maxResult = pg_query($conn, $maxQuery);

					if (!$maxResult) {
						// Output error message if max calculation fails
						echo "<div class='section'><p class='error-color'>An error occurred while fetching the max ma_inc: " . pg_last_error($conn) . " </p></div>";
						return;
					}

					$maxRow = pg_fetch_row($maxResult);
					$newMaInc = $maxRow[0] + 1; // Assign next available number

					// Define the SQL query for inserting a new employee record
					$query = "INSERT INTO t_mitarbeiter (vorname, nachname, ma_inc) VALUES ($1, $2, $3)";

					// Execute the query using parameterized inputs to prevent SQL injection
					$result = pg_query_params($conn, $query, array($firstname, $lastname, $newMaInc));

					// Check if the query execution was successful
					if (!$result) {
						// Output error message with error details
						echo "<div class='section'><p class='error-color'>An error occurred: " . pg_last_error($conn) . " </p></div>";
					} else {
						// Output success message
						echo "<div class='section'><p class='success-color'>The employee " . $firstname . " " . $lastname . " was successfully created.</p></div>";
					}
				}
			?>
		</div>
	</div>
	<?php
		echo file_get_contents ("footer.html");
	?>
  </body>
</html>

<h3 style="text-align: center;"><b>remove-employee.php</b></h3>
<ul>    
    <li>
    This PHP page lets users remove selected employees by checking their names from a list, deletes them from a PostgreSQL database, and displays success or error messages accordingly.
    </li>
</ul>

In [ ]:
<!-- Designed by Francisco Javier Cabello Rueda on 19/05/2025  -->

<?php
require_once "config/dbconfig.php";

$errorMessage = ''; // Variable to store error message in case no user is selected

if (isset($_POST['submit_btn'])) { // Check if the submit button is clicked
    if (isset($_POST['mitarbeiter'])) { // Check if any employee checkboxes are checked
        $mitarbeiter_namen = $_POST['mitarbeiter']; // Get array of selected employee last names
        $mitarbeiterstring = ""; // Initialize an empty string to store names for feedback
        
        foreach ($mitarbeiter_namen as $mitarbeiter) {
            if ($mitarbeiterstring != "")
                $mitarbeiterstring = $mitarbeiterstring . "," . $mitarbeiter; // Concatenate names with commas
            else
                $mitarbeiterstring = $mitarbeiter; // First name, no comma needed

            removeEmployee($mitarbeiter); // Perform the deletion operation
        }

        header('Location:?success=' . urlencode($mitarbeiterstring)); // Redirect to same page with feedback in URL
        exit();
    } else {
        $errorMessage = "Please select at least one employee to remove."; // Set error message if no user selected
    }
}

// Function to delete employees based on last names provided
function removeEmployee($name) {
    global $conn;
    $query = "DELETE FROM t_mitarbeiter WHERE nachname = $1"; // Prepare SQL query to delete by last name
    $result = pg_query_params($conn, $query, array($name)); // Execute query securely

    if (!$result) {
        echo "An error occurred: " . pg_last_error($conn); // Output any error encountered during execution
    }
}
?>
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <link rel="icon" type="image/png" sizes="32x32" href="assets/favicon/rewe.png"/>
        <title>Schichtplan | Remove-Employee</title>
        <link rel="stylesheet" href="assets/css/_header.css">
        <link rel="stylesheet" href="assets/css/_content.css">
        <link rel="stylesheet" href="assets/css/_footer.css">
        <link rel="stylesheet" href="assets/css/_font.css">
    </head>
    <body>
        <?php 
            echo file_get_contents("header.html"); 
        ?>
        <div class="content">
            <h2>Remove Employee</h2>
            <div class="container drop-shadow-m">
                <div class="line"></div>
                <form method="post">
                    <div class="section">
                        <p class="label">Please check the employees, that you want to remove.</p>
                    </div>
                    <div class="section">
                        <?php
                            $sql = "SELECT * FROM t_mitarbeiter ORDER BY nachname;"; // Query to fetch all employees ordered by last name
                            $result = pg_query($conn, $sql);

                            if ($result && pg_num_rows($result) > 0) {
                                echo '<table class="employee-table">'; // Start the HTML table for employees
                                $counter = 0; // Counter for employees in a row
                                
                                while ($row = pg_fetch_assoc($result)) {
                                    if ($counter % 4 == 0) { // Every four employees, start a new row
                                        if ($counter > 0) {
                                            echo '</tr>'; // Close previous row, if applicable
                                        }
                                        echo '<tr>'; // Start new row for the table
                                    }
                                    
                                    echo '<td>'; // Start table cell
                                    echo "<input type='checkbox' name='mitarbeiter[]' value=\"" . htmlspecialchars($row["nachname"]) . "\" class='employee-checkbox'>";
                                    echo "<label class='employee-label'>" . htmlspecialchars($row["nachname"]) . ", " . htmlspecialchars($row["vorname"]) . "</label>";
                                    echo '</td>'; // Close table cell

                                    $counter++; // Increment counter
                                }
                                echo '</tr>'; // Close last row
                                echo '</table>'; // Close the table
                            } else {
                                // No employees found in the database
                                echo "<div class='section'><p class='error-color'>No employees found in the database.</p></div>";
                            }
                        ?>
                    </div>
                    <div class="section">
                        <input type="submit" name="submit_btn" class="button-primary" value="Remove selected employees">
                    </div>
                </form>
                <?php
                    if ($errorMessage) { // Output error message if set
                        echo "<div class='section'><p class='error-color'>$errorMessage</p></div>";
                    }
                    
                    if (isset($_GET['success'])) { // Check if there is a 'success' parameter in the URL
                        $array_success = explode(",", $_GET['success']); // Convert the list of names into an array using comma separators
                        $mitarbeiter = implode(', ', array_map('htmlspecialchars', $array_success)); // Prepare names for safe display
                        $counter = count($array_success); // Count number of employees removed
                        
                        if ($counter == 1) {
                            echo "<div class='section'><p class='success-color'>The employee $mitarbeiter was successfully removed.</p></div>";
                        } else {
                            echo "<div class='section'><p class='success-color'>The employees $mitarbeiter were successfully removed.</p></div>";
                        }
                    }
                ?>
            </div>
        </div>
        <?php 
            echo file_get_contents("footer.html"); 
        ?>
    </body>
</html>

<div>
    <h2 style="text-align: center;"><b>5º SHIFTS ACTIONS</b></h2>
    <ul>
        <li>
            Now we will proceed to create the codes that will allow us to create and delete shifts from the database.
        </li>
        <br>
    </ul>
     <h3 style="text-align: center;"><b>create-shift.php</b></h3>
     <ul>
        <li>
            This PHP webpage lets users select employees, define a date range, choose a shift type, and create shift schedules, storing them in a PostgreSQL database while providing feedback on the success of the operation.
        </li>
     </ul>  
</div>

In [ ]:
<!-- Designed by Francisco Javier Cabello Rueda on 20/05/2025  -->

<?php
require_once "config/dbconfig.php"; 
?>

<!DOCTYPE html>
<html lang="en">
	<head>
		<meta charset="utf-8">
		<meta name="viewport" content="width=device-width, initial-scale=1.0">
		<link rel="icon" type="image/png" sizes="32x32" href="assets/favicon/rewe.png"/>
		<title>Schichtplan | Create-Shift</title>
		<link rel="stylesheet" href="assets/css/_header.css">
		<link rel="stylesheet" href="assets/css/_content.css">
		<link rel="stylesheet" href="assets/css/_footer.css">
		<link rel="stylesheet" href="assets/css/_font.css">
	</head>
	<body>
		<?php 
			echo file_get_contents("header.html"); 
		?>
		<div class="content">
			<h2>Create Shifts</h2>
			<div class="container drop-shadow-m">
				<div class="line"></div>
				<form method="post">
					<div class="section">
						<p class="label">Check one or more employees, enter a from and until date and select a shift type to create a shift(s).</p>
						<?php
							$sql = "SELECT * FROM t_mitarbeiter ORDER BY nachname;"; // Query to fetch all employees ordered by last name
                            $result = pg_query($conn, $sql);

                            if ($result && pg_num_rows($result) > 0) {
                                echo '<table class="employee-table">'; // Start the HTML table for employees
                                $counter = 0; // Counter for employees in a row
                                
                                while ($row = pg_fetch_assoc($result)) {
                                    if ($counter % 4 == 0) { // Every four employees, start a new row
                                        if ($counter > 0) { 
                                            echo '</tr>'; // Close previous row, if applicable
                                        }
                                        echo '<tr>'; // Start new row for the table
                                    }

                                    echo '<td>'; // Start table cell
                                    echo "<input type='checkbox' name='mitarbeiter[]' value=\"" . htmlspecialchars($row["nachname"]) . "\" class='employee-checkbox'>";
                                    echo "<label class='employee-label'>" . htmlspecialchars($row["nachname"]) . ", " . htmlspecialchars($row["vorname"]) . "</label>";
                                    echo '</td>'; // Close table cell
                                    
                                    $counter++; // Increment counter
                                }
                                echo '</tr>'; // Close last row
                                echo '</table>'; // Close the table
                            } else {
                                // No employees found in the database
                                echo "<div class='section'><p class='error-color'>No employees found in the database.</p></div>";
                            }
                    	?>
					</div>
					<div class="section">
						<span>Start: <input type="date" class="textfield_shift" name="date" value="<?php echo date("Y-m-d"); ?>" required></span>
						<span>End: <input type="date" class="textfield_shift" name="dateEnd" value="<?php echo date("Y-m-d"); ?>" required></span>
					</div>
					<div class="section">
						<table class="employee-table" style="width: 100%; border-collapse: collapse; text-align: center;">
							<tr>
								<td style="width: 25%; padding: 10px; text-align: center; vertical-align: middle;">
									<input type="radio" class="employee-checkbox ff" name="schichtart" value="ff" required>
									<label for="ff">FF (Früh-Früh)</label>
								</td>
								<td style="width: 25%; padding: 10px; text-align: center; vertical-align: middle;">
									<input type="radio" class="employee-checkbox ss" name="schichtart" value="ss" required>
									<label for="ss"> SS (Spät-Spät) </label>
								</td>
								<td style="width: 25%; padding: 10px; text-align: center; vertical-align: middle;">
									<input type="radio" class="employee-checkbox nn" name="schichtart" value="nn" required>
									<label for="nn">NN (Nacht-Nacht)</label>
								</td>
							</tr>
							<tr>
								<td style="width: 25%; padding: 10px; text-align: center; vertical-align: middle;">
									<input type="radio" class="employee-checkbox fs" name="schichtart" value="fs" required>
									<label for="fs">FS (Früh-Spät)</label>
								</td>
								<td style="width: 25%; padding: 10px; text-align: center; vertical-align: middle;">
									<input type="radio" class="employee-checkbox sn" name="schichtart" value="sn" required>
									<label for="sn">SN (Spät-Nacht)</label>
								</td>
								<td style="width: 25%; padding: 10px; text-align: center; vertical-align: middle;">
									<input type="radio" class="employee-checkbox nx" name="schichtart" value="nx" required>
									<label for="nx"> NX (Nacht-Frei) </label>
								</td>
							</tr>
						</table>					
					</div>
					<div class="section">
						<input type="submit" name="submit_btn" class="button-primary" value="Create shift">
					</div>
				</form>
				<?php
					// Check if the form submit button has been pressed
					if (isset($_POST['submit_btn'])) {
						// Gather inputs from the form
						$schicht = $_POST['schichtart'];
						$schichtBuchstabe = str_split($schicht); // Split the shift type into individual characters
						$schichtvar = ""; // Initialize the variable for the shift pattern
						$date = strtotime($_POST['date']); // Convert the start date into a timestamp
						$dateEnd = strtotime($_POST['dateEnd']); // Convert the end date into a timestamp
						$datediff = round(($dateEnd - $date) / (60 * 60 * 24)) + 1; // Calculate the number of days between the start and end dates

						// Verify that the start date is earlier or equal to the end date
						if ($date <= $dateEnd) {
							// Check if employees have been selected
							if (isset($_POST['mitarbeiter'])) {
								$mitarbeiter = $_REQUEST['mitarbeiter']; // Retrieve the list of selected employees
								foreach ($mitarbeiter as $name) { // Iterate over each selected employee
									// If the difference between dates is one day
									if ($datediff == 1) {
										// Insert values into the database for a single day
										schichtplanInsert(date('Y-m-d', $date), $schichtBuchstabe[0], $name);
									} else {
										// Logic to calculate shiftvar based on datediff and schichtvar logic
										if ($schichtvar == "") { // If the shift variable is empty
											$fill = array( // Define shift patterns based on selected type
												"ff" => "ffssnnxxxx",
												"ss" => "ssnnxxxx",
												"nn" => "nnxxxx",
												"fs" => "fssnnxxxx",
												"sn" => "snnxxxx",
												"nx" => "nxxxx"
											);
											$reihenfolge = array("f", "f", "s", "s", "n", "n", "x", "x", "x", "x"); // Define sequence for repetition

											$fillBuchstabe = str_split($fill[$schicht]); // Split selected pattern into characters
											// If the datediff is less than or equal to the size of the pattern
											if ($datediff <= sizeof($fillBuchstabe)) {
												for ($i = 0; $i < $datediff; $i++) {
													$schichtvar .= $fillBuchstabe[$i]; // Build shift variable with the pattern
												}
											} else {
												for ($i = 0; $i < sizeof($fillBuchstabe); $i++) {
													$schichtvar .= $fillBuchstabe[$i]; // Build shift variable until the pattern size
												}
												$remaining = $datediff - sizeof($fillBuchstabe); // Calculate remaining days after the first pattern
												for ($i = 0; $i < $remaining; $i++) {
													$schichtvar .= $reihenfolge[$i % sizeof($reihenfolge)]; // Cycle the sequence for the remaining days
												}
											}
										}
										$schichtvarBuchstabe = str_split($schichtvar); // Split the final shift sequence
										for ($i = 0; $i < $datediff; $i++) { // Iterate over each day in the range
											$newdate = strtotime("+" . $i . " day", $date); // Calculate the new date by incrementing days
											// Insert only if the character isn't 'x'
											if ($schichtvarBuchstabe[$i] != "x") {
												schichtplanInsert(date('Y-m-d', $newdate), $schichtvarBuchstabe[$i], $name); // Insert the shift into the database
											}
										}
									}
								}
							} else {
								// Error message if no employee is selected
								echo "<div class='section'><p class='error-color'>You haven't selected at least one employee.</p></div>";
								return;
							}
						} else {
							// Error message if the start date is newer than the end date
							echo "<div class='section'><p class='error-color'>The start date is newer than the end date.</p></div>";
							return;
						}

						// Create a string of selected employee names to display
						$namestr = implode(", ", array_map('htmlspecialchars', $mitarbeiter));
						echo "<div class='section'><p class='success-color'>The shift of " . $namestr . " has successfully been created.</p></div>";
					}

					// Method to insert data into the database
					function schichtplanInsert($date, $schicht, $name) {
						global $conn;
						$maxIdQuery = "SELECT MAX(_id) FROM t_schichtplan";
						$maxIdResult = pg_query($conn, $maxIdQuery);
						$maxIdRow = pg_fetch_row($maxIdResult);
						$nextId = $maxIdRow[0] + 1; // Increment the maximum id found

						// Define the SQL query for inserting a new shift record
						$query = "INSERT INTO t_schichtplan (_id, _datum, _schicht, _name) VALUES ($1, $2, $3, $4)";

						// Execute the query using parameterized inputs
						$result = pg_query_params($conn, $query, array($nextId, $date, $schicht, $name));
						
						if (!$result) {
							echo "An error occurred: " . pg_last_error($conn);
						}
					}
				?>
			</div>
		</div>
		<?php 
			echo file_get_contents("footer.html"); 
		?>
	</body>
</html>

<h3 style="text-align: center;"><b>remove-shift.php</b></h3>
<ul>
    <li>
        This PHP webpage allows users to select employees, specify a date range, and remove shift records from a PostgreSQL database, while displaying success or error messages based on the operation's outcome.
    </li>
</ul>  

In [ ]:
<!-- Designed by Francisco Javier Cabello Rueda on 21/05/2025  -->

<?php
	require_once "config/dbconfig.php"; 
?>

<!DOCTYPE html>
<html lang="en">
	<head>
		<meta charset="utf-8">
		<meta name="viewport" content="width=device-width, initial-scale=1.0">
		<link rel="icon" type="image/png" sizes="32x32" href="assets/favicon/rewe.png"/>
		<title>Schichtplan | Remove-Shift</title>
		<link rel="stylesheet" href="assets/css/_header.css">
		<link rel="stylesheet" href="assets/css/_content.css">
		<link rel="stylesheet" href="assets/css/_footer.css">
		<link rel="stylesheet" href="assets/css/_font.css">
	</head>
	<body>
		<?php 
			echo file_get_contents("header.html"); 
		?>
		<div class="content">
			<h2>Remove Shifts</h2>
			<div class="container drop-shadow-m">
				<div class="line"></div>
				<form method="post">
					<div class="section">
						<p class="label">Check one or more employees and enter a from and until date to remove shift(s).</p>
						<?php
							$sql = "SELECT * FROM t_mitarbeiter ORDER BY nachname;"; // Query to fetch all employees ordered by last name
                            $result = pg_query($conn, $sql);

                            if ($result && pg_num_rows($result) > 0) {
                                echo '<table class="employee-table">'; // Start the HTML table for employees
                                $counter = 0; // Counter for employees in a row
                                
                                while ($row = pg_fetch_assoc($result)) {
                                    if ($counter % 4 == 0) { // Every four employees, start a new row
                                        if ($counter > 0) { 
                                            echo '</tr>'; // Close previous row, if applicable
                                        }
                                        echo '<tr>'; // Start new row for the table
                                    }

                                    echo '<td>'; // Start table cell
                                    echo "<input type='checkbox' name='mitarbeiter[]' value=\"" . htmlspecialchars($row["nachname"]) . "\" class='employee-checkbox'>";
                                    echo "<label class='employee-label'>" . htmlspecialchars($row["nachname"]) . ", " . htmlspecialchars($row["vorname"]) . "</label>";
                                    echo '</td>'; // Close table cell
                                    
                                    $counter++; // Increment counter
                                }
                                echo '</tr>'; // Close last row
                                echo '</table>'; // Close the table
                            } else {
                                // No employees found in the database
                                echo "<div class='section'><p class='error-color'>No employees found in the database.</p></div>";
                            }
                    	?>
					</div>
					<div class="section">
						<span>Start: <input type="date" class="textfield_shift" name="date" value="<?php echo date("Y-m-d"); ?>" required></span>
						<span>End: <input type="date" class="textfield_shift" name="dateEnd" value="<?php echo date("Y-m-d"); ?>" required></span>
					</div>
					<div class="section">
						<input type="submit" name="submit_btn" class="button-primary" value="Remove shift">
					</div>
				</form>
					<?php
						// Was the submit button clicked?
						if (isset($_POST['submit_btn'])) {
							// Gather all inputs from the form
							$date = strtotime($_POST['date']);
							$dateEnd = strtotime($_POST['dateEnd']);

							// Check if the selected end date is later or equal to the start date
							if ($date <= $dateEnd) {
								// Check if at least one employee checkbox was checked
								if (isset($_POST['mitarbeiter'])) {
									$mitarbeiter = $_REQUEST['mitarbeiter'];
									// Loop through each selected employee
									foreach ($mitarbeiter as $name) {
										// Delete entries from the database
										schichtplanDelete($name, date('Y-m-d', $date), date('Y-m-d', $dateEnd));
									}
								} else {
									// Error message if no employee was selected
									echo "<div class='section'><p class='error-color'>You haven't selected at least one employee.</p></div>";
									return;
								}
							} else {
								// Error message if start date is newer than end date
								echo "<div class='section'><p class='error-color'>The start date is newer than the end date.</p></div>";
								return;
							}

							// Success message for database entry
							$namestr = "";
							foreach ($mitarbeiter as $name) {
								if ($namestr == "") {
									$namestr = $namestr . $name;
								} else {
									$namestr = $namestr . ", " . $name;
								}
							}
							if ($date == $dateEnd) {
								echo "<div class='section'><p class='success-color'>The shift of " . $namestr . " on " . date('d.m.y', $date) . " was successfully removed!</p></div>";
							} else {
								echo "<div class='section'><p class='success-color'>The shift of " . $namestr . " was successfully removed from " . date('d.m.y', $date) . " until " . date('d.m.y', $dateEnd) . ".</p></div>";
							}
						}

						// Method to delete data from the database
						function schichtplanDelete($name, $date, $dateEnd) {
							global $conn;
							$query = "DELETE FROM t_schichtplan WHERE _name = $1 AND _datum BETWEEN $2 AND $3;";
							// Execute deletion query with parameters
							$result = pg_query_params($conn, $query, array($name, $date, $dateEnd));
							if (!$result) {
								// Display an error message if the deletion fails
								echo "An error occurred: " . pg_last_error($conn);
							}
						}
					?>
			</div>
		</div>
		<?php 
			echo file_get_contents("footer.html"); 
		?>
	</body>
</html>

<div>
    <h2 style="text-align: center;"><b>6º LANGUAGES</b></h2>
    <ul>
        <li>
            Finally we will internationalise the application with 3 languages (English, Spanish and German) to be able to use it in any language, for this we will first create the lang.php file, then we will create langconfig.php y finally we will configure all the scripts.
        </li>
        <br>
    </ul>
     <h3 style="text-align: center;"><b>lang.php</b></h3>
     <ul>
        <li>
            This code starts a session, sets the user's preferred language in the session based on a URL parameter or defaults to English, and then redirects the user to the main page to apply the language change.
        </li>
     </ul>  
</div>

In [ ]:
<!-- Designed by Francisco Javier Cabello Rueda on 19/05/2025  -->
 
<?php
session_start(); // Start or resume the current session
// Set the session variable 'lang' to the value of the 'l' query parameter if available, otherwise default to 'en' (English)
$_SESSION['lang'] = $_GET['l'] ?? 'en';
// Get the previous page URL from the HTTP referrer
$previousPage = $_SERVER['HTTP_REFERER'] ?? './'; // Default to the root if referrer is not set
// Redirect the user to the previous page to maintain their navigation history
header("Location: $previousPage");
?>

<h3 style="text-align: center;"><b>langconfig.php</b></h3>
    <ul>
        <li>
            This code starts a session for managing language preferences, determines the user's selected language stored in the session (defaulting to English if none is set), checks for a corresponding language file, and loads it (or defaults to English if the file doesn't exist), parsing its contents for configuration settings (this code will be saved in the config folder).
        </li>
    </ul>  

In [ ]:
<?php
// Start session for language handling
session_start();

// Load language from the .ini file
$language = $_SESSION['lang'] ?? 'en'; // Default to English if not set
$file     = file_exists("languages/$language.ini") ? "languages/$language.ini" : "languages/en.ini";
$words    = parse_ini_file($file, true);
?>

<ul>
    <li>
        Finally we will create a folder called languages where we will add the files en.ini, es.ini and de.ini that will contain the respective configurations for each language. To apply them to the files we will have to call the langconfig.php file in all of them and modify the fragments we want to translate by the name of their variables.
    </li>
</ul>  


In [ ]:
<?php

//example for calling langconfig.php
require_once "config/langconfig.php";

//example for en.ini
[index]
home="Home"
welcome="Welcome to the Shift Plan!"
select_environment="Please select the database environment where you want to work:"

//example for translation
<h2><?php echo $words['index']['home']; ?></h2>
<p class="label"><?php echo $words['index']['welcome']; ?></p>
<p class="label"><?php echo $words['index']['select_environment']; ?></p>